In [153]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
%cd /content/drive/MyDrive/MyPython/ProbSpace/src

/content/drive/MyDrive/MyPython/ProbSpace/src


In [155]:
!pip install sklego

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [156]:
# Library
import os
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels
import statsmodels.api as sm
import lightgbm as lgb
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
from sklearn.feature_selection import RFE
from sklearn.preprocessing import FunctionTransformer
from sklego.preprocessing import RepeatingBasisFunction
from tqdm.auto import tqdm

import warnings
warnings.simplefilter('ignore')

In [157]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
seed = 42

In [158]:
train_df = pd.read_csv('../data/train_data.csv')
test_df = pd.read_csv('../data/test_data.csv')
submission_df = pd.read_csv('../data/submission.csv')

In [159]:
def preprocessing(df, mode='train'):
    df_tmp = df.copy()
    input_year = 2021

    df_tmp['time'] = pd.to_datetime(df_tmp.date, format='%m/%d')
    df_tmp['year'] = df_tmp['time'].dt.year
    df_tmp['month'] = df_tmp['time'].dt.month
    df_tmp['day'] = df_tmp['time'].dt.day
    
    if mode=='train':
        df_tmp.loc[df_tmp['month']>3, 'year'] = input_year
        df_tmp.loc[df_tmp['month']<=3, 'year'] = input_year + 1
    else:
        df_tmp['year'] = input_year + 1
    df_tmp['time'] = pd.to_datetime({'year':df_tmp.year, 'month':df_tmp.month, 'day':df_tmp.day})
    df_tmp['weekday'] = df_tmp['time'].dt.weekday
    df_tmp['day_of_year'] = df_tmp['time'].dt.day_of_year
    return df_tmp

train_df = preprocessing(train_df, mode='train')
test_df = preprocessing(test_df, mode='test')

In [160]:
train_df.columns

Index(['id', 'date', 'highest', 'lowest', 'rain', 'ice1', 'ice2', 'ice3',
       'oden1', 'oden2', 'oden3', 'oden4', 'hot1', 'hot2', 'hot3', 'dessert1',
       'dessert2', 'dessert3', 'dessert4', 'dessert5', 'drink1', 'drink2',
       'drink3', 'drink4', 'drink5', 'drink6', 'alcol1', 'alcol2', 'alcol3',
       'snack1', 'snack2', 'snack3', 'bento1', 'bento2', 'bento3', 'bento4',
       'tild1', 'tild2', 'men1', 'men2', 'men3', 'men4', 'men5', 'men6',
       'time', 'year', 'month', 'day', 'weekday', 'day_of_year'],
      dtype='object')

In [161]:
target_columns = ['ice1', 'ice2', 'ice3',
    'oden1', 'oden2', 'oden3', 'oden4', 'hot1', 'hot2', 'hot3', 'dessert1',
    'dessert2', 'dessert3', 'dessert4', 'dessert5', 'drink1', 'drink2',
    'drink3', 'drink4', 'drink5', 'drink6', 'alcol1', 'alcol2', 'alcol3',
    'snack1', 'snack2', 'snack3', 'bento1', 'bento2', 'bento3', 'bento4',
    'tild1', 'tild2', 'men1', 'men2', 'men3', 'men4', 'men5', 'men6']

In [162]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [163]:
# # sin,cos変換を適用
# def sin_transformer(period):
#     return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

# def cos_transformer(period):
#     return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

# all_df_sincos = all_df.copy()
# all_df_sincos['day_of_year'] = all_df.time.dt.day_of_year

# # month -> sin, cos
# all_df_sincos['month_sin'] = sin_transformer(12).fit_transform(all_df_sincos['month'])
# all_df_sincos['month_cos'] = cos_transformer(12).fit_transform(all_df_sincos['month'])

# # week -> sin, cos
# all_df_sincos['weekday_sin'] = sin_transformer(7).fit_transform(all_df_sincos['weekday'])
# all_df_sincos['weekday_cos'] = cos_transformer(7).fit_transform(all_df_sincos['weekday'])

# # day -> sin, cos
# all_df_sincos['day_sin'] = sin_transformer(365).fit_transform(all_df_sincos['day_of_year'])
# all_df_sincos['day_cos'] = cos_transformer(365).fit_transform(all_df_sincos['day_of_year'])


# fig, ax = plt.subplots(3, 1, sharex=True, figsize=(16,8))
# all_df_sincos[["month_sin", "month_cos"]].plot(ax=ax[0])
# all_df_sincos[["weekday_sin", "weekday_cos"]].plot(ax=ax[1])
# all_df_sincos[["day_sin", "day_cos"]].plot(ax=ax[2])
# plt.suptitle("Cyclical encoding with sine/cosine transformation")

In [164]:
# # 放射状基底関数
# all_df_sincos_rbf = all_df_sincos.copy()

# rbf = RepeatingBasisFunction(n_periods=12, column='day_of_year', input_range=(1, 365), remainder='drop')
# rbf.fit(all_df_sincos_rbf)

# tmp_column = [f'month_rbf_{i}' for i in range(1, 13)]
# day_of_year_rbf = pd.DataFrame(columns=tmp_column, data=rbf.transform(all_df_sincos_rbf))

# # 放射状基底関数を結合
# all_df_sincos_rbf = pd.merge(all_df_sincos_rbf, day_of_year_rbf, left_index=True, right_index=True)

# # プロット
# day_of_year_rbf.plot(subplots=True, figsize=(14, 8),
#      	sharex=True, title="Radial Basis Functions",
#      	legend=False)

In [165]:
# # train, test に分ける
# train_df_sincos_rbf = all_df_sincos_rbf.iloc[:350, :]
# test_df_sincos_rbf = all_df_sincos_rbf.iloc[350:, :].dropna(axis=1)

In [166]:
# 検証データのindexを指定(訓練データの最後2ヶ月を使用)
# valid_index = range(297,351) # month: 2,3
valid_index = range(325,351) # month: 3
# valid_index = range(1, 21) # month: 4

In [167]:
# 予測結果を保存する辞書型データ
results = dict({})
all_lgb_score = []

# 商品ごとの予測を作成

for c in tqdm(target_columns):
    # train_tmp = train_df_sincos_rbf.copy()
    train_tmp = train_df.copy()
    # test_tmp = test_df_sincos_rbf.copy()
    test_tmp = test_df.copy()
    
    #ice
    if c in ['ice1', 'ice2', 'ice3']:
        # 予測期間はアイスが人気な7, 8 ,9月ではないので除外
        train_tmp = train_tmp[~train_tmp['month'].isin([7, 8, 9])]
        # アイスは金曜に人気
        train_tmp['is_wday4'] = train_df['weekday'].isin([4]).astype(int)
        test_tmp['is_wday4'] = test_df['weekday'].isin([4]).astype(int)
    # oden
    elif c in ['oden1', 'oden2', 'oden3', 'oden4']:
        # おでんやって無い夏は考えない
        train_tmp = train_tmp[~train_tmp['month'].isin([7, 8, 9])]
        # おでんは水木で人気
        train_tmp['is_wday23'] = train_df['weekday'].isin([2, 3]).astype(int)
        test_tmp['is_wday23'] = test_df['weekday'].isin([2, 3]).astype(int)
    # hot
    elif c in ['hot1', 'hot2', 'hot3']:
        # ホットスナックは月、火、金で不人気
        train_tmp['is_wday014'] = train_df['weekday'].isin([0,1,4]).astype(int)
        test_tmp['is_wday014'] = test_df['weekday'].isin([0,1,4]).astype(int)
    # dessert
    elif c in ['dessert1', 'dessert2', 'dessert3', 'dessert4', 'dessert5']:
        # デザートは水曜と日曜で人気
        train_tmp['is_wday36'] = train_df['weekday'].isin([3,6]).astype(int)
        test_tmp['is_wday36'] = test_df['weekday'].isin([3,6]).astype(int)
    # drink1234
    elif c in ['drink1', 'drink2', 'drink3', 'drink4']:
        # 夏に人気なドリンクは夏のデータは使わない
        train_tmp = train_tmp[~train_tmp['month'].isin([7, 8, 9])]
        if c in ['drink1', 'drink4']:
            # 金曜に人気
            train_tmp['is_wday4'] = train_df['weekday'].isin([4]).astype(int)
            test_tmp['is_wday4'] = test_df['weekday'].isin([4]).astype(int)
        else:
            # 常に人気が変わらないドリンクは全期間つかう。火曜に人気
            train_tmp['is_wday1'] = train_df['weekday'].isin([1]).astype(int)
            test_tmp['is_wday1'] = test_df['weekday'].isin([1]).astype(int)
    # drink5
    elif c in ['drink5']:
        # 金曜に人気
        train_tmp['is_wday4'] = train_df['weekday'].isin([4]).astype(int)
        test_tmp['is_wday4'] = test_df['weekday'].isin([4]).astype(int)
    # drink6
    elif c in ['drink6']:
        # 日曜に人気
        train_tmp['is_wday6'] = train_df['weekday'].isin([6]).astype(int)
        test_tmp['is_wday6'] = test_df['weekday'].isin([6]).astype(int)
    # alcohol
    elif c in ['alcol1', 'alcol2', 'alcol3']:
        # 酒は水木で人気
        train_tmp['is_wday23'] = train_df['weekday'].isin([2, 3]).astype(int)
        test_tmp['is_wday23'] = test_df['weekday'].isin([2, 3]).astype(int)
    # snack
    elif c in ['snack1', 'snack2', 'snack3']:
        train_tmp['is_wday0'] = train_df['weekday'].isin([0]).astype(int)
        train_tmp['is_wday14'] = train_df['weekday'].isin([1,4]).astype(int)
        test_tmp['is_wday0'] = test_df['weekday'].isin([0]).astype(int)
        test_tmp['is_wday14'] = test_df['weekday'].isin([1,4]).astype(int)
    # bento
    elif c in ['bento1', 'bento2', 'bento3', 'bento4']:
        # 弁当は月、火、金で不人気
        train_tmp['is_wday014'] = train_df['weekday'].isin([0,1,4]).astype(int)
        test_tmp['is_wday014'] = test_df['weekday'].isin([0,1,4]).astype(int)
    # tild
    elif c in ['tild1', 'tild2']:
        # 日曜に人気
        train_tmp['is_wday6'] = train_df['weekday'].isin([6]).astype(int)
        test_tmp['is_wday6'] = test_df['weekday'].isin([6]).astype(int)
    # men
    elif c in ['men1', 'men2', 'men3', 'men4', 'men5', 'men6']:
        train_tmp['is_wday014'] = train_df['weekday'].isin([0,1,4]).astype(int)
        test_tmp['is_wday014'] = test_df['weekday'].isin([0,1,4]).astype(int)
    
    # 訓練に確実に不必要なデータを除く
    train_columns = [c for c in train_tmp.columns if c not in target_columns if c not in\
    # ['id', 'date', 'time', 'month', 'year', 'day', 'weekday', 'day_of_year']]
        ['id', 'date', 'time']]
    
    # データを訓練、検証に分割
    X_train = train_tmp[~train_tmp['id'].isin(valid_index)][train_columns]
    y_train = train_tmp[~train_tmp['id'].isin(valid_index)][c]
    X_valid = train_tmp[train_tmp['id'].isin(valid_index)][train_columns]
    y_valid = train_tmp[train_tmp['id'].isin(valid_index)][c]
     
    # テストデータ
    X_test = test_tmp[train_columns]

    # 分位点
    qs = np.array([0.01, 0.1, 0.5, 0.9, 0.99])
    
    # 分位点ごとのスコア
    q_scores = []
    
    # 分位点ごとにモデルを作成
    for q in qs:
        lgb = LGBMRegressor(
            objective='quantile',
            alpha = q,
            n_estimators=10000,
            colsample_bytree=0.9,
            random_state=seed,
            max_depth=5,
        )
        
        # # RFEにより特徴量選択
        # selector = RFE(lgb, n_features_to_select=0.6)
        # selector.fit(X_train, y_train)
        
        # # 特徴量が生き残ったか（残った：True、ドロップ：False）を表示
        # result = pd.DataFrame(selector.get_support(), index=X_train.columns.values, columns=['left'])
        # result['ranking'] = selector.ranking_
        # print(f'---{c}_{q}---')
        # display(result.T)
        # print(' ')
        
        # このモデルで採用されたカラム
        # train_columns_RFE = X_train.columns.values[selector.get_support()]
        
        # # 採用されたカラムだけでデータを作成
        # X_train_RFE = X_train[train_columns_RFE]
        # X_valid_RFE = X_valid[train_columns_RFE]
        # X_test_RFE = X_test[train_columns_RFE]
        
        
        # 学習
        lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=False)
        score = lgb.best_score_['valid_0']['quantile']
        
        y_pred = lgb.predict(X_test)
        results[(c, q)] = y_pred
        
        q_scores.append(score)

    all_lgb_score.append(q_scores)

score_df_1 = pd.DataFrame(np.array(all_lgb_score), columns=qs, index=target_columns)
display(score_df_1)

print(f'ave score: {np.array(all_lgb_score).mean()}')

  0%|          | 0/39 [00:00<?, ?it/s]

,0.01,0.10,0.50,0.90,0.99
ice1,0.038482,0.259937,0.835238,0.601046,0.085618
ice2,0.188182,0.256613,0.516754,0.894251,0.292401
ice3,0.052765,0.437846,1.380013,0.455547,0.080848
oden1,0.494649,1.033110,2.813888,1.507459,0.236395
oden2,0.272256,1.107961,3.061858,1.236656,0.275408
oden3,0.663922,2.167868,3.844927,2.752824,0.681132
oden4,0.239680,2.023314,4.720410,4.071914,1.083020
hot1,0.313659,1.213945,3.369788,1.556435,0.640527
hot2,0.277802,2.148717,6.163041,2.737138,0.330684
hot3,0.333115,2.438793,5.467799,2.333935,0.317607


ave score: 0.7485971195170656


In [168]:
submit_rows = [[f'{k[0]}_{k[1]}'] + v.tolist() for k, v in results.items()]

In [188]:
# submitファイルを作成
# データを整数に丸めることでわずかに精度が上がるらしい
submit_df_1 = pd.DataFrame(np.array(submit_rows)[:, 1:22].astype(float), index=np.array(submit_rows)[:, 0], columns=range(1, 22))
submit_df_1_round = pd.DataFrame(np.array(submit_rows)[:, 1:22].astype(float).round(), index=np.array(submit_rows)[:, 0], columns=range(1, 22))

# 転置
submit_df_1_round = submit_df_1_round.transpose()
submit_df_1 = submit_df_1.transpose()

submit_df_1.index.name = 'id'
submit_df_1_round.index.name = 'id'

submit_df_1_round.to_csv("../work/submission/sub_lgb_kotrying_plus_day_of_year_valid3_max_depth5_true_index.csv")
display(submit_df_1_round)

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.0,19.0,21.0,22.0,28.0,34.0,60.0,63.0,63.0,69.0,24.0,26.0,30.0,34.0,37.0,6.0,33.0,40.0,45.0,72.0,33.0,44.0,52.0,63.0,91.0,65.0,91.0,105.0,131.0,182.0,44.0,53.0,64.0,68.0,144.0,104.0,138.0,157.0,175.0,206.0,195.0,226.0,228.0,241.0,261.0,186.0,215.0,240.0,254.0,270.0,20.0,45.0,50.0,54.0,57.0,36.0,64.0,75.0,81.0,81.0,7.0,9.0,53.0,57.0,57.0,26.0,58.0,61.0,64.0,67.0,45.0,58.0,75.0,82.0,87.0,24.0,32.0,39.0,38.0,43.0,21.0,31.0,33.0,36.0,45.0,11.0,21.0,22.0,24.0,43.0,27.0,38.0,40.0,45.0,50.0,22.0,23.0,25.0,26.0,27.0,7.0,8.0,8.0,10.0,16.0,48.0,55.0,60.0,62.0,82.0,25.0,44.0,46.0,51.0,126.0,51.0,54.0,56.0,60.0,67.0,17.0,22.0,28.0,31.0,35.0,9.0,17.0,20.0,23.0,37.0,27.0,35.0,42.0,45.0,58.0,42.0,45.0,51.0,54.0,63.0,37.0,37.0,44.0,48.0,50.0,45.0,60.0,72.0,79.0,78.0,27.0,26.0,28.0,30.0,34.0,4.0,11.0,12.0,16.0,19.0,9.0,9.0,10.0,15.0,15.0,31.0,42.0,52.0,60.0,62.0,25.0,27.0,30.0,35.0,35.0,28.0,31.0,37.0,45.0,49.0,29.0,35.0,39.0,50.0,59.0,29.0,32.0,37.0,45.0,50.0,13.0,34.0,38.0,48.0,61.0
2,16.0,18.0,20.0,21.0,28.0,34.0,56.0,55.0,59.0,69.0,24.0,26.0,30.0,32.0,37.0,6.0,25.0,25.0,37.0,72.0,31.0,36.0,27.0,49.0,91.0,56.0,81.0,76.0,87.0,182.0,44.0,46.0,44.0,57.0,144.0,104.0,136.0,149.0,153.0,197.0,193.0,213.0,230.0,232.0,258.0,186.0,207.0,225.0,248.0,259.0,19.0,28.0,32.0,32.0,34.0,36.0,49.0,53.0,57.0,61.0,5.0,7.0,10.0,11.0,18.0,26.0,30.0,31.0,37.0,49.0,33.0,34.0,38.0,47.0,46.0,24.0,26.0,28.0,31.0,41.0,21.0,27.0,29.0,34.0,45.0,11.0,17.0,19.0,22.0,28.0,27.0,34.0,35.0,39.0,49.0,22.0,23.0,25.0,26.0,26.0,7.0,7.0,8.0,9.0,16.0,72.0,66.0,69.0,75.0,87.0,39.0,57.0,64.0,69.0,126.0,63.0,63.0,73.0,77.0,81.0,17.0,29.0,33.0,39.0,38.0,9.0,27.0,33.0,36.0,38.0,27.0,41.0,51.0,61.0,67.0,9.0,

In [170]:
results = dict({})
all_lgb_score_2 = []

for c in tqdm(target_columns):
    
    # ある商品の全スコア 
    q_scores = []
    
    # 21日個別のモデルを作る
    for i in range(1, 22):
        # print(f'{c}_{i}')
        
        all_tmp = all_df.copy()
    
        # おでんの場合は7, 8, 9月を考えない
        if c in ['oden1', 'oden2', 'oden3', 'oden4']:
            # おでんやって無い夏は考えない
            all_tmp = all_tmp[~all_tmp['month'].isin([7, 8, 9])]
        
        if i <= 7:
            # 10週のラグ特徴量を考える
            for j in range(1, 11):
                all_tmp[f'lag_{j}week'] = np.log1p(all_tmp[c].shift(7*j))
                # ラグ特徴量の5週間の平均
            all_tmp[f'lag_ave_5week'] = all_tmp[[f'lag_{k}week'for k in range(1, 6)]].mean(axis=1)
            
        elif i > 7 and i <= 14:
            for j in range(1, 11):
                all_tmp[f'lag_{j+1}week'] = np.log1p(all_tmp[c].shift(7*(j+1)))
            all_tmp[f'lag_ave_5week'] = all_tmp[[f'lag_{k}week'for k in range(2, 7)]].mean(axis=1)
        elif i > 14:
            for j in range(1, 11):
                all_tmp[f'lag_{j+2}week'] = np.log1p(all_tmp[c].shift(7*(j+2)))
            all_tmp[f'lag_ave_5week'] = all_tmp[[f'lag_{k}week'for k in range(3, 8)]].mean(axis=1)
        
        # 最新20日のデータ
        cnt = 1
        while cnt <= 20:
            if (cnt+i-1) % 7 == 0:
                cnt += 1
                continue
            all_tmp[f'last_{cnt}day'] = np.log1p(all_tmp[c].shift(cnt+i-1))
            cnt += 1
        
        # データ分割 + 訓練データの最初のNaNをけす
        train_tmp = all_tmp[all_tmp.index<350]
        train_tmp = train_tmp.dropna(axis=0)
        
        test_tmp = all_tmp[all_tmp.index==350+i-1]
        test_tmp = test_tmp.dropna(axis=1)
        
        # 除く特徴量
        removed_cols = ['date', 'id', 'weekday', 'year', 'month', 'day_of_year', 'day', 'time']
        train_columns = [col for col in train_tmp.columns if col not in target_columns if col not in removed_cols]
        X_train = train_tmp[~train_tmp['id'].isin(valid_index)][train_columns]
        y_train = train_tmp[~train_tmp['id'].isin(valid_index)][c]
        X_valid = train_tmp[train_tmp['id'].isin(valid_index)][train_columns]
        y_valid = train_tmp[train_tmp['id'].isin(valid_index)][c]
        
        X_test = test_tmp[train_columns]
    
        
        qs = np.array([0.01, 0.1, 0.5, 0.9, 0.99])
    
        # 分位点ごとのスコア
        i_q_scores = []
        
       
        # 分位点ごとにモデルを作成
        for q in qs:
            lgb = LGBMRegressor(
                objective='quantile',
                alpha = q,
                n_estimators=10000,
                colsample_bytree=0.9,
                random_state=seed,
                max_depth=5,
            )
            
            lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=False)
            
            # 商品c, i日目, 分位点q のスコア
            score = lgb.best_score_['valid_0']['quantile']
        
            y_pred = lgb.predict(X_test)
            results[(i, c, q)] = y_pred

            # 商品c, i日目のスコア配列
            i_q_scores.append(score)

        q_scores.append(i_q_scores)

    q_scores = np.array(q_scores)
        
    # ある商品のqごとのスコア(21日を平均)
    q_score = np.mean(q_scores, axis=0)
        
    all_lgb_score_2.append(list(q_score))

score_df_2 = pd.DataFrame(np.array(all_lgb_score_2), columns=qs, index=target_columns)
display(score_df_2)

print(f'ave score: {np.array(all_lgb_score_2).mean()}')
                  

  0%|          | 0/39 [00:00<?, ?it/s]

,0.01,0.10,0.50,0.90,0.99
ice1,0.038365,0.298515,0.904014,0.636897,0.192059
ice2,0.105779,0.550402,1.060660,0.852312,0.346556
ice3,0.053437,0.430499,1.219799,0.478735,0.136503
oden1,0.417963,1.842837,3.212259,1.436948,0.404921
oden2,0.368496,1.772154,3.380320,3.280573,0.754788
oden3,0.699890,3.900505,6.446681,5.056110,0.910254
oden4,0.480469,1.900883,4.750980,3.744463,0.998008
hot1,0.318682,1.896238,4.559971,3.033460,0.717326
hot2,0.361659,2.898281,7.323455,3.249730,0.417855
hot3,0.336060,2.596433,6.452747,3.281155,0.437856


ave score: 0.9737912486524712


In [190]:
# 提出ファイル作成
submit_df_2 = submission_df.copy()
submit_df_2_round = submission_df.copy()
for i in range(21):
    for c in target_columns:
        for q in qs:
            submit_df_2.loc[i, f'{c}_{q}'] = results[(i+1, c, q)]
            submit_df_2_round.loc[i, f'{c}_{q}'] = results[(i+1, c, q)].round()

submit_df_2_round = submit_df_2_round.set_index('id')
submit_df_2 = submit_df_2.set_index('id')

In [177]:
submit_df_2_round.to_csv("../work/submission/sub_lgb_lag_first_valid3_remove_timeinfo.csv")
display(submit_df_2_round)

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.0,20.0,20.0,21.0,51.0,44.0,62.0,61.0,64.0,88.0,24.0,26.0,28.0,33.0,34.0,12.0,23.0,39.0,53.0,108.0,29.0,43.0,48.0,53.0,90.0,45.0,88.0,96.0,161.0,200.0,34.0,46.0,56.0,65.0,147.0,101.0,134.0,149.0,195.0,203.0,181.0,199.0,220.0,254.0,266.0,187.0,208.0,223.0,256.0,271.0,19.0,38.0,51.0,53.0,54.0,36.0,39.0,71.0,70.0,88.0,5.0,43.0,51.0,61.0,59.0,26.0,52.0,62.0,64.0,67.0,32.0,60.0,62.0,72.0,75.0,14.0,31.0,36.0,41.0,72.0,18.0,31.0,34.0,39.0,47.0,9.0,19.0,21.0,20.0,22.0,27.0,37.0,40.0,44.0,73.0,22.0,23.0,24.0,26.0,29.0,8.0,8.0,8.0,9.0,16.0,56.0,57.0,59.0,61.0,96.0,40.0,47.0,51.0,55.0,127.0,37.0,54.0,56.0,61.0,108.0,20.0,22.0,25.0,32.0,33.0,11.0,17.0,19.0,18.0,27.0,27.0,38.0,44.0,47.0,57.0,8.0,47.0,50.0,53.0,61.0,12.0,35.0,45.0,46.0,57.0,17.0,63.0,71.0,78.0,80.0,7.0,26.0,29.0,31.0,31.0,4.0,6.0,6.0,15.0,18.0,7.0,8.0,10.0,14.0,18.0,17.0,48.0,53.0,62.0,64.0,15.0,26.0,31.0,32.0,36.0,11.0,34.0,36.0,42.0,54.0,26.0,34.0,39.0,47.0,47.0,4.0,36.0,41.0,46.0,50.0,13.0,32.0,38.0,48.0,71.0
2,16.0,17.0,18.0,21.0,51.0,44.0,52.0,58.0,57.0,101.0,24.0,26.0,28.0,33.0,41.0,14.0,31.0,26.0,38.0,94.0,29.0,40.0,35.0,49.0,89.0,67.0,85.0,88.0,166.0,192.0,38.0,51.0,49.0,63.0,146.0,101.0,125.0,157.0,190.0,223.0,180.0,194.0,226.0,247.0,266.0,189.0,205.0,227.0,245.0,271.0,19.0,20.0,23.0,38.0,41.0,36.0,40.0,47.0,57.0,75.0,5.0,6.0,9.0,15.0,42.0,25.0,27.0,29.0,35.0,65.0,32.0,33.0,37.0,49.0,74.0,14.0,21.0,25.0,28.0,90.0,17.0,26.0,28.0,37.0,57.0,9.0,15.0,19.0,15.0,47.0,27.0,31.0,33.0,35.0,72.0,22.0,23.0,24.0,26.0,28.0,7.0,8.0,8.0,9.0,16.0,63.0,73.0,73.0,75.0,90.0,44.0,60.0,61.0,79.0,127.0,37.0,64.0,69.0,76.0,108.0,20.0,26.0,32.0,37.0,38.0,13.0,28.0,33.0,35.0,35.0,27.0,40.0,51.0,57.0,68.0,8.0,

In [185]:
submit_df_1

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.115362,19.070461,20.823487,22.036640,28.042009,34.000000,60.369848,63.323544,63.253236,68.519566,24.040,26.10,30.489326,33.660929,36.870776,6.000000,33.388901,39.617592,44.770817,72.221182,33.426205,44.382040,51.908647,63.476593,91.038923,64.985100,90.816165,105.356586,131.436172,182.450590,44.335922,53.320712,63.592707,68.123431,144.002007,103.748334,138.198939,156.709470,175.414786,206.264595,194.790089,226.380724,228.210301,240.894398,261.257917,186.487045,215.482302,240.145390,254.067967,269.850072,19.935706,45.289069,50.487174,54.267888,56.897322,36.102,63.750522,74.818217,80.698787,81.017532,6.761304,8.641957,53.379048,57.015335,56.985004,25.937672,58.314304,60.745622,64.486427,66.781033,45.188182,57.644098,75.275761,81.580242,86.876237,24.207418,32.127296,39.436973,38.040968,43.476687,20.986679,31.059279,32.684229,35.912099,44.751661,10.713195,20.640236,22.214367,23.560392,43.001924,27.100,38.165124,40.366980,44.500348,50.042180,22.1,22.975287,24.6645,25.973219,26.596881,7.261976,7.729678,8.000921,9.501923,16.00000,47.756406,55.438416,59.593735,62.143760,81.790730,24.873740,44.309386,46.292798,51.119193,126.408995,51.086016,54.107840,56.029198,59.788206,67.150984,17.0,22.344804,27.554766,31.001918,34.597847,9.1,16.604366,19.517129,23.170145,36.896281,27.386258,34.506777,42.012985,45.134039,58.000000,41.936603,44.840699,50.502901,54.298147,63.032660,37.020926,36.819898,43.963929,47.667707,49.555341,44.917359,60.112579,71.855446,78.905410,77.530607,26.670738,25.938301,27.637069,29.958826,33.526496,4.100,11.036326,12.305946,16.477925,19.260531,9.432012,9.058568,10.417646,14.834141,15.000000,30.652805,42.277941,51.942951,59.546468,61.655433,2

In [191]:
submit_df_2

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.308677,19.769432,20.447681,21.268188,50.892066,43.562287,61.792115,61.254908,64.351141,88.427808,24.100000,25.800000,27.865562,33.125355,34.066855,11.877838,22.678555,39.330975,53.483177,108.326999,29.329676,43.389079,48.394201,53.048083,89.820093,45.179067,88.456514,96.149567,161.241368,199.718885,34.427134,45.524539,56.166039,65.341125,147.326004,101.437174,133.959201,149.018730,194.643271,203.189124,180.969530,198.518922,220.007193,253.690505,266.388111,187.060196,207.935495,223.329391,256.103080,270.826041,19.100,37.947773,50.947152,53.264365,53.633336,36.102,38.813000,70.978039,70.155575,87.638632,5.100000,43.328495,51.101348,60.899703,58.533071,26.094473,52.222701,61.709111,64.273793,67.033937,32.040000,59.687250,62.119922,71.968193,75.241741,13.782120,31.457960,36.322015,41.433192,71.803806,17.988430,30.552401,33.629432,39.205007,46.897137,9.3,19.055396,20.993211,20.094503,21.885371,27.100000,36.562354,40.293339,43.668847,73.074513,22.000000,22.760523,24.222844,26.190842,28.581042,7.509369,8.024473,8.065493,9.379575,16.000000,55.840664,57.383837,58.672011,60.520846,95.774889,39.922039,47.258758,51.049541,55.433178,126.958012,36.962359,54.434592,55.598481,61.494763,108.115000,19.697113,22.331171,25.282593,32.173407,33.250728,11.425713,17.107134,18.756331,18.399073,27.026745,26.685994,37.723222,43.992786,46.531499,57.422788,8.100000,46.576538,50.193225,53.365476,60.502318,11.510565,34.856052,44.810400,46.045965,57.382223,16.929477,62.664674,70.873264,77.825885,79.627773,7.473632,26.461724,28.905621,30.704918,31.215814,4.082088,6.265724,6.489044,14.800000,17.874446,6.995976,8.388884,10.330124,13.560986,18.0,17.100000,48.085376,52.685808,62.0

In [196]:
# アンサンブル
df_ans = (submit_df_1 + submit_df_2) / 2
df_ans = df_ans.round()
df_ans

,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,16.0,19.0,21.0,22.0,39.0,39.0,61.0,62.0,64.0,78.0,24.0,26.0,29.0,33.0,35.0,9.0,28.0,39.0,49.0,90.0,31.0,44.0,50.0,58.0,90.0,55.0,90.0,101.0,146.0,191.0,39.0,49.0,60.0,67.0,146.0,103.0,136.0,153.0,185.0,205.0,188.0,212.0,224.0,247.0,264.0,187.0,212.0,232.0,255.0,270.0,20.0,42.0,51.0,54.0,55.0,36.0,51.0,73.0,75.0,84.0,6.0,26.0,52.0,59.0,58.0,26.0,55.0,61.0,64.0,67.0,39.0,59.0,69.0,77.0,81.0,19.0,32.0,38.0,40.0,58.0,19.0,31.0,33.0,38.0,46.0,10.0,20.0,22.0,22.0,32.0,27.0,37.0,40.0,44.0,62.0,22.0,23.0,24.0,26.0,28.0,7.0,8.0,8.0,9.0,16.0,52.0,56.0,59.0,61.0,89.0,32.0,46.0,49.0,53.0,127.0,44.0,54.0,56.0,61.0,88.0,18.0,22.0,26.0,32.0,34.0,10.0,17.0,19.0,21.0,32.0,27.0,36.0,43.0,46.0,58.0,25.0,46.0,50.0,54.0,62.0,24.0,36.0,44.0,47.0,53.0,31.0,61.0,71.0,78.0,79.0,17.0,26.0,28.0,30.0,32.0,4.0,9.0,9.0,16.0,19.0,8.0,9.0,10.0,14.0,17.0,24.0,45.0,52.0,61.0,63.0,20.0,27.0,30.0,34.0,35.0,20.0,32.0,36.0,44.0,52.0,27.0,35.0,39.0,49.0,53.0,17.0,34.0,39.0,46.0,50.0,13.0,33.0,38.0,48.0,66.0
2,16.0,18.0,19.0,21.0,40.0,39.0,54.0,57.0,58.0,85.0,24.0,26.0,29.0,33.0,39.0,10.0,28.0,26.0,38.0,83.0,30.0,38.0,31.0,49.0,90.0,61.0,83.0,82.0,127.0,187.0,41.0,48.0,47.0,60.0,145.0,102.0,131.0,153.0,172.0,210.0,187.0,203.0,228.0,240.0,262.0,188.0,206.0,226.0,246.0,265.0,19.0,24.0,27.0,35.0,37.0,36.0,45.0,50.0,57.0,68.0,5.0,6.0,10.0,13.0,30.0,26.0,29.0,30.0,36.0,57.0,33.0,34.0,37.0,48.0,60.0,19.0,23.0,26.0,30.0,65.0,19.0,26.0,28.0,35.0,51.0,10.0,16.0,19.0,18.0,37.0,27.0,32.0,34.0,37.0,60.0,22.0,23.0,24.0,26.0,27.0,7.0,7.0,8.0,9.0,16.0,67.0,69.0,71.0,75.0,89.0,42.0,59.0,62.0,74.0,127.0,50.0,64.0,71.0,77.0,95.0,19.0,27.0,32.0,38.0,38.0,11.0,28.0,33.0,35.0,37.0,27.0,41.0,51.0,59.0,67.0,8.

In [197]:
df_ans.to_csv("../work/submission/sub_lgb_lag_first_valid3_remove_timeinfo_and_lgbbase_ansamble.csv")

In [173]:
# # 目視確認
# pred_median_col = [c for c in submit_df.columns if '_0.5' in c]
# test_add_pred = test_df.merge(submit_df[pred_median_col].reset_index(), on='id', how='left')
# test_add_pred.columns = [c.replace('_0.5', '') if '_0.5' in c else c for c in test_add_pred.columns]
# test_add_pred[target_columns] = test_add_pred[target_columns].astype(float)
# test_add_pred.head(3)

In [174]:
# # 移動平均
# window=7
# train_rolling = train_df.rolling(window, min_periods=1).mean()
# test_rolling = test_add_pred.rolling(window, min_periods=1).mean()
# all_rolling = pd.concat([train_df, test_add_pred], axis=0).reset_index(drop=True).rolling(window, min_periods=1).mean()

In [175]:
# # 目視確認
# plot_col = [c for c in train_df.columns if c not in ['id', 'date', 'time', 'year', 'month', 'day', 'weekday', 'day_of_year']]
# ncols = len(plot_col) // 13
# plt.subplots(14, ncols, sharey=True, sharex=True, figsize=(30, 80))
# for i, col in enumerate(plot_col):
#     plt.subplot(14, ncols, i+1)
#     plt.plot(train_rolling.index[window:], train_rolling[col][window:], alpha=1, color='blue', label='train')
#     plt.plot(all_rolling.index[-len(test_rolling):], all_rolling[col][-len(test_rolling):], alpha=1, color='red', label='test')
#     for x in [20,51,81,112,143,173.204,234,265,296,324,350]:
#         plt.axvline(x)
#     plt.xlabel(col)
#     plt.legend()
#     plt.xticks([])
# plt.show()